<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Code Explainer

In this lecture we are going to build a code explainer that will create a docstring of a given function! 

## Imports

In [1]:
import inspect  # Used to transform python code to a string
import os
import openai

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [2]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Prompt design
As we want to obtain explanation about what some piece of code does, as well as a list of the possible bugs, we might want to use two separate prompts and later combine the result:

In [4]:
def docstring_prompt(code):
    prompt = f"{code}\n # A high quality python docstring of the above python function:\n\"\"\""
    return prompt

In [5]:
def hello(name):
    print(f'Hello {name}')

In [6]:
print(inspect.getsource(hello))

def hello(name):
    print(f'Hello {name}')



In [7]:
print(docstring_prompt(inspect.getsource(hello)))

def hello(name):
    print(f'Hello {name}')

 # A high quality python docstring of the above python function:
"""


## OpenAI API Call
Note that we now use the *code-davinci-002* model instead of text-davinci-003.

In [8]:
response = openai.Completion.create(
  model="code-davinci-002",
  prompt=docstring_prompt(inspect.getsource(hello)),
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\"\"\""]

)


In [9]:
print(response["choices"][0]["text"])


This is a function that prints hello to the name provided

Args:
    name (str): The name of the person you want to greet

Returns:
    None



## Combine docstring and function

In [10]:
def merge_docstring_and_function(original_function, docstring):
    function_string = inspect.getsource(original_function)
    split = function_string.split("\n")
    first_part, second_part = split[0], split[1:]
    # note the spaces before the three "
    merged_function = first_part + "\n" + '    """' + docstring + '    """' + "\n" + "\n".join(second_part)
    return merged_function

In [11]:
merged_code = merge_docstring_and_function(hello, response["choices"][0]["text"])

In [12]:
print(merged_code)

def hello(name):
    """
This is a function that prints hello to the name provided

Args:
    name (str): The name of the person you want to greet

Returns:
    None
    """
    print(f'Hello {name}')



## Get the docstring of a more complicated function

In [13]:
def test_function(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view


In [14]:
response = openai.Completion.create(
  model="code-davinci-002",
  prompt=docstring_prompt(inspect.getsource(test_function)),
  temperature=0,
  max_tokens=256,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\"\"\""] # Corresponds to """, the end of the docstring

)


In [15]:
merged_code = merge_docstring_and_function(test_function, response["choices"][0]["text"])
print(merged_code)

def test_function(test, num_questions):
    """
    This function takes in a string of a test and returns a dictionary of the test with the question number as the key and the question as the value.
    :param test: a string of a test
    :param num_questions: the number of questions in the test
    :return: a dictionary of the test with the question number as the key and the question as the value
    """
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view



In [16]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers


In [17]:
response = openai.Completion.create(
  model="code-davinci-002",
  prompt=docstring_prompt(inspect.getsource(take)),
  temperature=0,
  max_tokens=256,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\"\"\""] # Corresponds to """, the end of the docstring

)


In [18]:
merged_code = merge_docstring_and_function(take, response["choices"][0]["text"])
print(merged_code)

def take(student_view):
    """
    Takes a student view of the exam and returns a dictionary of answers.
    :param student_view: A dictionary of questions and their corresponding views.
    :return: A dictionary of answers.
    """
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers

